In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pywt import dwt2, idwt2

In [47]:
# calling cover image
A = cv2.imread('image.jpg')
host = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)
LL, (LH, HL, HH) = dwt2(host, 'haar')
h, w = LL.shape
# print('h,w', h, w)

# calling watermark image
# B = cv2.imread('barbara512.jpg') # try watermark image. text to png
B = cv2.imread('watermark2.png') # try watermark image. text to png
watermark = cv2.cvtColor(B, cv2.COLOR_BGR2GRAY)
LL_w, (LH_w, HL_w, HH_w) = dwt2(watermark, 'haar')
hh, ww = LL_w.shape
# print('hh,ww', hh, ww)

# compute xoff and yoff for placement of upper left corner of resized image
# ---- center ----
# yoff = round((h-hh)/2)
# xoff = round((w-ww)/2)

# --- top left ----
# yoff = 0
# xoff = 0

# --- bottom left ---
# yoff = h-hh
# xoff = 0

# --- top right ---
# yoff = 0
# xoff = w-ww

# --- bottom right ---
yoff = h-hh
xoff = w-ww

# print('yoff,xoff', yoff, xoff)

# adding watermark
manipulated = LL_w * 0.98
# use numpy indexing to place the resized image in the center of background image
result = LL.copy()
result[yoff:yoff+hh, xoff:xoff+ww] += manipulated
# new_host_LL = LL + manipulated 
result_LL = idwt2((result,( LH, HL, HH)), 'haar')
cv2.imwrite('watermarked.jpeg', result_LL)


# extracting watermark
wm_LL,( m_LH, wm_HL, m_HH) = dwt2(result_LL, 'haar')
new_LL = wm_LL - LL
new_LL = new_LL / 0.98
new_image = idwt2((new_LL, (LH, HL, HH)), 'haar')
new_crop_image = new_image.copy()
cv2.imwrite('before_extracted_watermark.jpeg', new_crop_image)
# new_crop_image = new_crop_image[h-hh:h+hh, w-ww:w+ww] # center
# new_crop_image = new_crop_image[0:hh*2, 0:ww*2] # top left
# new_crop_image = new_crop_image[h*2-hh*2:h*2, 0:ww*2] # bottom left
# new_crop_image = new_crop_image[0:hh*2, w*2-ww*2:w*2+ww*2] # top right
new_crop_image = new_crop_image[h*2-hh*2:h*2, w*2-ww*2:w*2+ww*2] # bottom right
cv2.imwrite('extracted_watermark.jpeg', new_crop_image)

# extracting image
wm_LL,( m_LH, wm_HL, m_HH) = dwt2(result_LL, 'haar')
new_LL = wm_LL.copy()
new_LL[yoff:yoff+hh, xoff:xoff+ww] -= manipulated
new_LL = new_LL 
new_image = idwt2((new_LL, (LH, HL, HH)), 'haar')
cv2.imwrite('extracted_image.jpeg', new_image)

(3648, 5472)
1824 2736
79 373


True